# Midterm Expectation

- Baseline
- Experiment (i.e., something different; e.x., spreadsheet)
- Timeline (what we have done, and what we will be doing)

# Import Dependencies

In [ ]:
import os

import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import (
    CenterCrop,
    Compose,
    ToTensor,
)
from PIL import Image
from torch.utils.data import DataLoader, Dataset

# Global Variables

In [ ]:
DATA_PATH = "./../sample_data"
METADATA_DIR = f"{DATA_PATH}/metadata"
IMAGERY_DIR = f"{DATA_PATH}/imagery/realsense_overhead"

# Helper Functions for Data

In [ ]:
def read_csv_variable_cols(filepath: str) -> pd.DataFrame:
    """https://stackoverflow.com/a/57824142.
    We only read the first 6 columns to retrieve required labels.
    """
    ### Loop the data lines
    with open(filepath, 'r') as temp_f:
        # get No of columns in each line
        col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

    ### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
    column_names = [i for i in range(0, max(col_count))]

    ### Read csv
    return pd.read_csv(filepath, header=None, delimiter=",", names=column_names, low_memory=False).iloc[:,:6]

# Ingredient and Dish Metadata (Groun Truths)

## Data Format

### Training-testing data

- "imagery/realsense_overhead/dish_<id>" contains the images as the input data
- "dish_ids" contains training-testing splits

### Labels (metadata)

All labels need to be preprocessed. For each dish, we need to extract the following:
- total calorie
- mass (optional according to the paper)
- the amount of the three macronutrients (fat, carb, protein)

It doesn't seem that bad - we don't need to process the ingredients because they are purely there for constructing the labels. For our multi-task learning, we only need to have the above labels. The three tasks are:

1. Calorie
2. Macronutrients (fat, carb, protein)
3. Mass (optional)

这样一来我们可以把labels和image data放在一起，每次返回input和expected output.

## Ingredients Metadata (deprecated)

In [ ]:
ingredient_metadata = pd.read_csv(f"{METADATA_DIR}/ingredients_metadata.csv", names=["name", "id", "cal", "fat", "carb", "protein"], skiprows=1)
print(ingredient_metadata)

## Dish Metadata

In [ ]:
# Metadata for dishes has variable numbers of columns per row.
# Can do similar stuff to dish_metadata_cafe2.csv
# The first 6 columns: [dish_id, total_calories, total_mass, total_fat, total_carb, total_protein]
dish_metadata_1 = read_csv_variable_cols(f"{METADATA_DIR}/dish_metadata_cafe1.csv")
# Rename the columns
dish_metadata_1 = dish_metadata_1.rename(columns={0:"dish_id", 1:"total_calories", 2:"total_mass", 3:"total_fat", 4:"total_carb", 5:"total_protein"})
print(dish_metadata_1.iloc[:10,:])

# Convert to dictionary
labels_dict = dish_metadata_1.set_index("dish_id").to_dict("index")
print(labels_dict)

# Datasets and DataLoaders

In [ ]:
class RGBDataset(Dataset):
    """4.2 The input resolution to the
    network is a 256x256 image, where images were downsized
    and center cropped in order to retain the most salient dish
    region.

    我们baseline应该只用RGB就行 (根据4.2).
    """

    # TODO: Also return the metadata here?
    def __init__(self, data_dir, transforms=Compose([CenterCrop((256, 256)), ToTensor()]), labels=labels_dict):
        self.data_dir = data_dir
        self.transforms = transforms
        self.labels = labels

        # # ['dish_1556572657', 'dish_1556573514', 'dish_1556575014', 'dish_1556575083', 'dish_1556575124', 'dish_1556575273', 'dish_1556575327']
        self.dish_ids = sorted(os.listdir(self.data_dir))

        self.img_paths = list(
            map(
                lambda fname: os.path.join(self.data_dir, fname),
                self.dish_ids,
            )
        )

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        rgd_path = f"{self.img_paths[idx]}/rgb.png"
        dish_id = self.dish_ids[idx]
        label = torch.tensor(list(self.labels[dish_id].values()))
        return self.transforms(Image.open(rgd_path)), label

In [ ]:
rgb_dataset = RGBDataset(IMAGERY_DIR)
for x, y in rgb_dataset:
    print(x, y)
    # torch.Size([3, 256, 256])
    break

# InceptionV2

In [ ]:
def ConvBNReLU(in_channels,out_channels,kernel_size,stride=1,padding=0):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride,padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU6(inplace=True),
    )

def ConvBNReLUFactorization(in_channels,out_channels,kernel_sizes,paddings):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_sizes, stride=1,padding=paddings),
        nn.BatchNorm2d(out_channels),
        nn.ReLU6(inplace=True)
    )

class InceptionV2ModuleA(nn.Module):
    def __init__(self, in_channels,out_channels1,out_channels2reduce, out_channels2, out_channels3reduce, out_channels3, out_channels4):
        super(InceptionV2ModuleA, self).__init__()

        self.branch1 = ConvBNReLU(in_channels=in_channels,out_channels=out_channels1,kernel_size=1)

        self.branch2 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels2reduce, kernel_size=1),
            ConvBNReLU(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_size=3, padding=1),
        )

        self.branch3 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels,out_channels=out_channels3reduce,kernel_size=1),
            ConvBNReLU(in_channels=out_channels3reduce, out_channels=out_channels3, kernel_size=3, padding=1),
            ConvBNReLU(in_channels=out_channels3, out_channels=out_channels3, kernel_size=3, padding=1),
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels4, kernel_size=1),
        )

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out4 = self.branch4(x)
        out = torch.cat([out1, out2, out3, out4], dim=1)
        return out

class InceptionV2ModuleB(nn.Module):
    def __init__(self, in_channels,out_channels1,out_channels2reduce, out_channels2, out_channels3reduce, out_channels3, out_channels4):
        super(InceptionV2ModuleB, self).__init__()

        self.branch1 = ConvBNReLU(in_channels=in_channels,out_channels=out_channels1,kernel_size=1)

        self.branch2 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels2reduce, kernel_size=1),
            ConvBNReLUFactorization(in_channels=out_channels2reduce, out_channels=out_channels2reduce, kernel_sizes=[1,3],paddings=[0,1]),
            ConvBNReLUFactorization(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_sizes=[3,1],paddings=[1, 0]),
        )

        self.branch3 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels,out_channels=out_channels3reduce,kernel_size=1),
            ConvBNReLUFactorization(in_channels=out_channels3reduce, out_channels=out_channels3reduce,kernel_sizes=[1, 3], paddings=[0, 1]),
            ConvBNReLUFactorization(in_channels=out_channels3reduce, out_channels=out_channels3reduce,kernel_sizes=[3, 1], paddings=[1, 0]),
            ConvBNReLUFactorization(in_channels=out_channels3reduce, out_channels=out_channels3reduce, kernel_sizes=[1, 3], paddings=[0, 1]),
            ConvBNReLUFactorization(in_channels=out_channels3reduce, out_channels=out_channels3,kernel_sizes=[3, 1], paddings=[1, 0]),
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels4, kernel_size=1),
        )

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out4 = self.branch4(x)
        out = torch.cat([out1, out2, out3, out4], dim=1)
        return out

class InceptionV2ModuleC(nn.Module):
    def __init__(self, in_channels,out_channels1,out_channels2reduce, out_channels2, out_channels3reduce, out_channels3, out_channels4):
        super(InceptionV2ModuleC, self).__init__()

        self.branch1 = ConvBNReLU(in_channels=in_channels,out_channels=out_channels1,kernel_size=1)

        self.branch2_conv1 = ConvBNReLU(in_channels=in_channels, out_channels=out_channels2reduce, kernel_size=1)
        self.branch2_conv2a = ConvBNReLUFactorization(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_sizes=[1,3],paddings=[0,1])
        self.branch2_conv2b = ConvBNReLUFactorization(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_sizes=[3,1],paddings=[1,0])

        self.branch3_conv1 = ConvBNReLU(in_channels=in_channels,out_channels=out_channels3reduce,kernel_size=1)
        self.branch3_conv2 = ConvBNReLU(in_channels=out_channels3reduce, out_channels=out_channels3, kernel_size=3,stride=1,padding=1)
        self.branch3_conv3a = ConvBNReLUFactorization(in_channels=out_channels3, out_channels=out_channels3, kernel_sizes=[3, 1],paddings=[1, 0])
        self.branch3_conv3b = ConvBNReLUFactorization(in_channels=out_channels3, out_channels=out_channels3, kernel_sizes=[1, 3],paddings=[0, 1])

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels4, kernel_size=1),
        )

    def forward(self, x):
        out1 = self.branch1(x)
        x2 = self.branch2_conv1(x)
        out2 = torch.cat([self.branch2_conv2a(x2), self.branch2_conv2b(x2)],dim=1)
        x3 = self.branch3_conv2(self.branch3_conv1(x))
        out3 = torch.cat([self.branch3_conv3a(x3), self.branch3_conv3b(x3)], dim=1)
        out4 = self.branch4(x)
        out = torch.cat([out1, out2, out3, out4], dim=1)
        return out

class InceptionV3ModuleD(nn.Module):
    def __init__(self, in_channels,out_channels1reduce,out_channels1,out_channels2reduce, out_channels2):
        super(InceptionV3ModuleD, self).__init__()

        self.branch1 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels1reduce, kernel_size=1),
            ConvBNReLU(in_channels=out_channels1reduce, out_channels=out_channels1, kernel_size=3,stride=2,padding=1)
        )

        self.branch2 = nn.Sequential(
            ConvBNReLU(in_channels=in_channels, out_channels=out_channels2reduce, kernel_size=1),
            ConvBNReLU(in_channels=out_channels2reduce, out_channels=out_channels2, kernel_size=3, stride=1, padding=1),
            ConvBNReLU(in_channels=out_channels2, out_channels=out_channels2, kernel_size=3, stride=2,padding=1),
        )

        self.branch3 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out = torch.cat([out1, out2, out3], dim=1)
        return out

class InceptionAux(nn.Module):
    def __init__(self, in_channels,out_channels):
        super(InceptionAux, self).__init__()

        self.auxiliary_avgpool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.auxiliary_conv1 = ConvBNReLU(in_channels=in_channels, out_channels=128, kernel_size=1)
        self.auxiliary_conv2 = nn.Conv2d(in_channels=128, out_channels=768, kernel_size=5,stride=1)
        self.auxiliary_dropout = nn.Dropout(p=0.7)
        self.auxiliary_linear1 = nn.Linear(in_features=768, out_features=out_channels)

    def forward(self, x):
        x = self.auxiliary_conv1(self.auxiliary_avgpool(x))
        x = self.auxiliary_conv2(x)
        x = x.view(x.size(0), -1)
        out = self.auxiliary_linear1(self.auxiliary_dropout(x))
        return out

class InceptionV2(nn.Module):
    def __init__(self, num_classes=1000, stage='train'):
        super(InceptionV2, self).__init__()
        self.stage = stage

        self.block1 = nn.Sequential(
            ConvBNReLU(in_channels=3, out_channels=64, kernel_size=7,stride=2,padding=3),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
        )

        self.block2 = nn.Sequential(
            ConvBNReLU(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2,padding=1),
        )

        self.block3 = nn.Sequential(
            InceptionV2ModuleA(in_channels=192,out_channels1=64,out_channels2reduce=64, out_channels2=64, out_channels3reduce=64, out_channels3=96, out_channels4=32),
            InceptionV2ModuleA(in_channels=256, out_channels1=64, out_channels2reduce=64, out_channels2=96,out_channels3reduce=64, out_channels3=96, out_channels4=64),
            InceptionV3ModuleD(in_channels=320, out_channels1reduce=128, out_channels1=160, out_channels2reduce=64,out_channels2=96),
        )

        self.block4 = nn.Sequential(
            InceptionV2ModuleB(in_channels=576, out_channels1=224, out_channels2reduce=64, out_channels2=96,out_channels3reduce=96, out_channels3=128, out_channels4=128),
            InceptionV2ModuleB(in_channels=576, out_channels1=192, out_channels2reduce=96, out_channels2=128,out_channels3reduce=96, out_channels3=128, out_channels4=128),
            InceptionV2ModuleB(in_channels=576, out_channels1=160, out_channels2reduce=128, out_channels2=160,out_channels3reduce=128, out_channels3=128, out_channels4=128),
            InceptionV2ModuleB(in_channels=576, out_channels1=96, out_channels2reduce=128, out_channels2=192,out_channels3reduce=160, out_channels3=160, out_channels4=128),
            InceptionV3ModuleD(in_channels=576, out_channels1reduce=128, out_channels1=192, out_channels2reduce=192,out_channels2=256),
        )

        self.block5 = nn.Sequential(
            InceptionV2ModuleC(in_channels=1024, out_channels1=352, out_channels2reduce=192, out_channels2=160,out_channels3reduce=160, out_channels3=112, out_channels4=128),
            InceptionV2ModuleC(in_channels=1024, out_channels1=352, out_channels2reduce=192, out_channels2=160,
                               out_channels3reduce=192, out_channels3=112, out_channels4=128)
        )

        self.avg_pool = nn.AdaptiveAvgPool2d((2,2))
        self.dropout = nn.Dropout(p=0.5)
    

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.avg_pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        return x

if __name__=='__main__':
    model = InceptionV2()
    print(model)

    input = torch.randn(1, 3, 256, 256)
    out = model(input)
    print(out.shape)

# Baseline Model

In [ ]:
class BaseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = InceptionV2([1, 1, 3])
        self.fc1 = nn.Linear(4096, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc_calories = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.Linear(4096, 1)
        )
        self.fc_mass = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.Linear(4096, 1)
        )
        self.fc_mc = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.Linear(4096, 3)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.fc2(self.fc1(x))

        x_cal = self.fc_calories(x)
        x_mass = self.fc_mass(x)
        x_mn = self.fc_mc(x)

        return x_cal, x_mass, x_mn

# Multi-task Loss

In [ ]:
class MultiTaskLearner(nn.Module):
    def __init__(self, model: nn.Module):
        super(MultiTaskLearner, self).__init__()
        self.model = model
        self.criterion = nn.L1Loss()

    def forward(self, x, y):
        # 1 x 5 Tensor [total_calories, total_mass, total_fat, total_carb, total_protein]
        # [0] - total_calories 

        loss_calorie = self.criterion(x[0], y[0])
        
        loss_mass = self.criterion(x[1], y[1])

        loss_mn = self.criterion(x[2], y[2])

        loss_total = loss_calorie + loss_mass + loss_mn

        return loss_total

In [ ]:
model = BaseNet()
learner = MultiTaskLearner(model)
input = torch.randn(1, 3, 256, 256)
out = model(input)
print(out[0].shape, out[1].shape, out[2].shape)

loss = learner(out, [torch.Tensor([[1]]),torch.Tensor([[2]]),torch.Tensor([[3,4,5]])])
loss.backward()